In [1]:
import numpy as np
import gurobipy as grb
import csv
import time
import statsmodels.api as sm
from joblib import Parallel,delayed

def five_variate(n,SNR,rho):
    beta1 = np.array([1,1,1,0,1])
    beta2 = np.array([-1,-1,-1,0,-1])
    p = len(beta1)
    Sigma = np.zeros((p,p));
    for i in range(p):
        for j in range(p):
            Sigma[i,j] = rho**(abs(i-j))
    X = np.random.multivariate_normal(mean =np.zeros(p), cov = Sigma, size = n)
    noise1 = np.random.normal(0,np.sqrt(np.var(X[:int(n/2)]@beta1)/SNR), size = int(n/2)) 
    noise2 = np.random.normal(0,np.sqrt(np.var(X[:int(n/2)]@beta2)/SNR), size = int(n/2)) 
    Y1 = X[:int(n/2)]@beta1 + noise1
    Y2 = X[int(n/2):]@beta2 + noise2
    Y = np.r_[Y1,Y2]
    lam = 0.1*n
    ols = sm.OLS(Y, X)
    ols_result = ols.fit()
    bigM = 10*float(1.96*(sum(ols_result.bse)))#1000

    M = grb.Model()
    M.Params.TimeLimit = 400
    beta = M.addMVar((int(n),5), lb = -grb.GRB.INFINITY)
    z = M.addMVar(len(Y) - 1, vtype = grb.GRB.BINARY)
    sum_of_sq = sum((Y[t] - sum(X[t,j]*beta[t,j] for j in range(5)))* (Y[t] - sum(X[t,j]*beta[t,j] for j in range(5))) for t in range(len(Y)))
    for t in range(len(Y)-1):
        for j in range(5):
            M.addConstr(beta[t+1,j] - beta[t,j] <= bigM*z[t])
            M.addConstr(beta[t+1,j] - beta[t,j] >= -bigM*z[t])
    #optional constratins to add more cuts
    M.addConstr(sum(z) <= 5)
    for t in range(1, len(Y) - 2):
        M.addConstr(z[t-1] + z[t] + z[t+1] <= 1)
    M.setObjective(sum_of_sq+ lam*sum(z))
    M.optimize()
    beta_hat = [beta[t].x for t in range(len(Y))]
    z_hat = [z[t].x for t in range(len(Y)-1)]
    opt_gap = M.MIPGap
    return z_hat, opt_gap,beta_hat

SNR_list = np.array([6, 3.52, 2.07, 1.22, 0.71])
rho_list = np.array([0, 0.3, 0.7])
n_list = np.array([1000])

header = ['Repitition','Time','rho', 'SNR', 'n', 'Optimality Gap', 'Change Points']#, 'beta_hat', 'nonzero count']
f = open('simulation_five_variate.csv', 'w')
writer = csv.writer(f)
writer.writerow(header)
f.close()

def to_repeat(rep):
    for n in n_list:
        for SNR in SNR_list:
            for rho in rho_list:
                tik = time.time()
                z_hat, opt_gap, beta_hat = five_variate(n,SNR,rho)
                tok = time.time()
                duration = tok - tik
                cps = [idx for idx,item in enumerate(z_hat) if item == 1 ]
                # Write result
                resultrow = [rep,duration,rho,SNR, n, opt_gap, cps]
                f = open('simulation_five_variate.csv', 'a')
                writer = csv.writer(f)
                writer.writerow(resultrow)
                f.close()
                
Parallel(n_jobs = 8)(delayed(to_repeat)(rep) for rep in range(500)) #Tomo's machine has 8 cores.

Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-07
Set parameter TimeLimit to value 400
Set parameter Username
Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-07
Academic license - for non-commercial use only - expires 2024-01-07
Set parameter TimeLimit to value 400
Set parameter TimeLimit to value 400
Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-07
Set parameter TimeLimit to value 400
Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-07
Set parameter TimeLimit to value 400
Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-07
Set parameter TimeLimit to value 400
Set parameter Username
Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-07
Academic license - for non-commercial use only - expires 2024-01-07
Set parameter TimeLimit to value 400
Set parameter


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   11436    2.3540250e+03   0.000000e+00   3.298094e+03      5s

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   11949    2.2794338e+03   0.000000e+00   3.416588e+03      5s

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   11083    2.3937316e+03   0.000000e+00   3.131338e+03      5s

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   11485    2.4214664e+03   0.000000e+00   3.244450e+03      5s
   15467    1.8760354e+03   0.000000e+00   2.899013e+03     10s
   23471    1.1944791e+03   0.000000e+00   1.448236e+03     10s
   16428    1.7971479e+03   0.000000e+00   2.355490e+03     10s
   16319    1.8378280e+03   0.000000e+00   2.595335e+03     10s
   15648    1.9096912e+03   0.000000e+00   2.427024e+03     10s
   16161    1.8366909e+03   0.000000e+00   2.538372e+03     10s
   15619    1.90

     3     8  615.68606    2  132 4448.32374  615.68606  86.2%   209   58s
     0     2  605.21220    0  123 4557.21979  605.21220  86.7%     -   59s
    25    36  601.41333    5  135 4304.96827  601.41333  86.0%   125   59s
     1     4  583.44012    1  138 4540.67908  583.44012  87.2%  28.0   59s
     7    16  624.25347    3  152 4537.18732  624.25347  86.2%   127   58s
    25    36  580.30485    5  117 4239.13370  580.30485  86.3%   161   60s
     0     2  655.19434    0  140 4902.08756  655.19434  86.6%     -   62s
    15    26  668.33586    4  147 4767.26724  668.33586  86.0%   119   63s
     7    16  616.60676    3  131 4448.32374  616.60676  86.1%   189   66s
     1     4  605.21220    1  120 4557.21979  605.21220  86.7%  48.0   67s
    15    26  627.67938    4  153 4537.18732  624.25347  86.2%   116   67s
    25    36  668.72394    5  146 4767.26724  668.72394  86.0%   103   69s
     1     4  655.47313    1  140 4902.08756  655.47313  86.6%  47.0   70s
     3     8  583.72827  

   493    33     cutoff   43       667.64134  609.52095  8.71%  52.8  277s
   160    34  699.61203   15   67  729.78414  603.61535  17.3%   159  277s
   103    36  768.63635   10  104  808.16587  670.82184  17.0%   167  278s
   158    30  714.97736   16   61  737.33374  621.97916  15.6%   148  278s
   151    28  674.83927   15   76  705.91445  595.63617  15.6%   167  277s
   114    10  719.62425    7  116  735.67381  653.04710  11.2%   127  279s
   541    34  609.96464   48   80  667.64134  609.96464  8.64%  50.7  281s
   114    27  740.58275   11   94  788.10946  664.51833  15.7%   171  283s
   166    39  674.85769   16   74  705.91445  595.63617  15.6%   158  284s
   171    36  699.69073   16   71  729.78414  603.61535  17.3%   156  284s
   585    36  610.33528   50   80  667.64134  610.33528  8.58%  49.3  285s
    55    33  607.59416    8  129  730.04772  606.58552  16.9%   100  285s
   178    38  714.98747   19   59  737.33374  621.97916  15.6%   137  286s
   114    38  769.07073  

  1333   503  705.58233   28  105  737.33374  651.10947  11.7%  46.3  395s
   378   124  772.24502   35   74  808.16587  671.87316  16.9%  86.4  396s
  1725     8     cutoff  219       667.64134  638.15143  4.42%  34.9  397s
  1310   444  688.43384  139   85  705.91445  595.63617  15.6%  50.5  397s
   783    67  714.79886   66   93  735.67381  658.95162  10.4%  50.9  397s
   154    34  704.98694   16   68  730.04772  606.58552  16.9%   143  398s
  1445   527  706.70175   40  103  737.33374  651.10947  11.7%  45.0  400s
  1775     5     cutoff  226       667.64134  639.06440  4.28%  34.7  400s
   436   154  773.41904   39   76  808.16587  671.87316  16.9%  79.3  400s
   172    40  705.01286   19   65  730.04772  606.58552  16.9%   132  400s

Explored 1546 nodes (105707 simplex iterations) in 400.25 seconds (84.20 work units)
Thread count was 20 (of 20 available processors)

Solution count 2: 737.334 4448.32 

Time limit reached
Best objective 7.373337367417e+02, best bound 6.51109470509

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[x86])

CPU model: Intel(R) Core(TM) i9-10910 CPU @ 3.60GHz
Thread count: 10 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 10988 rows, 5999 columns and 33960 nonzeros
Model fingerprint: 0xea60a28a
Model has 15000 quadratic objective terms
Variable types: 5000 continuous, 999 integer (999 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [2e-05, 1e+02]
  QObjective range [2e-09, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective 6818.2648706
Presolve time: 0.13s
Presolved: 10988 rows, 5999 columns, 33960 nonzeros
Presolved model has 15000 quadratic objective terms
Variable types: 5000 continuous, 999 integer (999 binary)
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[x86])

CPU model: I

   35999    8.0962334e+02   0.000000e+00   0.000000e+00     41s

Root relaxation: objective 8.096233e+02, 35999 iterations, 40.42 seconds (16.17 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  809.62334    0  224 6726.76609  809.62334  88.0%     -   40s
   36194    7.8432842e+02   0.000000e+00   0.000000e+00     41s

Root relaxation: objective 7.843284e+02, 36194 iterations, 40.75 seconds (16.35 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  784.32842    0  210 6470.09280  784.32842  87.9%     -   40s
   34409    8.5496868e+02   0.000000e+00   8.550508e+01     40s
     0     0  782.85803    0  188 5921.64333  782.85803  86.8%     -   41s
     0     0  742.38187    0  225 6635.16659  742.38187  88.8%     -   39s
     0     0  811.

KeyboardInterrupt: 

In [6]:
bigM

8.833158764852824